In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from random import seed
from random import randrange

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
real_estate = pd.read_csv('/content/drive/MyDrive/MachineLearning/datasets/Real_estate.csv')
real_estate.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [65]:
X = real_estate.iloc[:,1:-1].values
y = (real_estate.iloc[:,-1].values)
print(X)

[[2012.917     32.        84.87882   10.        24.98298  121.54024]
 [2012.917     19.5      306.5947     9.        24.98034  121.53951]
 [2013.583     13.3      561.9845     5.        24.98746  121.54391]
 ...
 [2013.25      18.8      390.9696     7.        24.97923  121.53986]
 [2013.         8.1      104.8101     5.        24.96674  121.54067]
 [2013.5        6.5       90.45606    9.        24.97433  121.5431 ]]


In [66]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)
print('n samples in Train set: ', len(X_train))
print('n samples in Test set: ', len(X_test))

n samples in Train set:  331
n samples in Test set:  83


#XGBoost#

*XGBoost is a Gradient Boosting algorithm. Similar to the idea behind the Adaboost with the difference that, instead of tweaking the instance weights at every iteration like AdaBoost does, this method tries to fit the new predictor to the residual errors made by the previous predictor. This is again a method where a number of predictors is added to an ensemble. Each predictor correct the previous one in order to create at the end a strong classifier*

In [70]:
import xgboost
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute

#train phase
#n_estimators = n. of trees 
#max_depth = max depth of each tree
#eta = learning rate
xgboost_model = xgboost.XGBRegressor(n_estimators=100, max_depth=5, eta=0.01,objective ='reg:squarederror')
#repeats K-Fold n times with different randomization in each repetition.
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [74]:
#test phase
scores = cross_val_score(xgboost_model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
#MAE: is the mean overseen data of the absolute differences between true and predicted values
#std: A standard deviation is a measure of how dispersed the data is in relation to the mean. Low standard deviation means data are clustered around the mean, and high standard deviation indicates data are more spread out
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 5.010 (0.744)


**Sample Prediction**

In [73]:
# make a prediction
#fit the model 
xgboost_model.fit(X_train,y_train)
#create a fake sample
fake_sample = [2013.25,23.5,104.81,3,24.98746,121.54391]
fake_sample = np.asarray([fake_sample])
y_pred = xgboost_model.predict(fake_sample)
# summarize prediction
print('Predicted output: %.3f' % y_pred)

Predicted output: 45.927
